<a href="https://colab.research.google.com/github/kuriousk516/HIST4916a-Stolen_Bronzes/blob/main/Stolen_Bronzes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stolen: The Benin Bronzes and Repatriation

## Introduction

>"*Walk into any European museum today and you will see the curated spoils of Empire. They sit behind plate glass: dignified, tastefully lit. Accompanying pieces of card offer a name, date and place of origin. They do not mention that the objects are all stolen*."
>
> 'Radicals in Conversation': The Brutish Museums

Public history and digital humanities offers a locus point of contending with difficult pasts. Museums, often considered bastions of knowledge, learning, and public good have fallen under an increasingly critical gaze -- and rightfully so. Public museums have been tools of colonialism, racism, and superiority centred around the supremacy of the west and its history. 

Digital repositories of museum archives and websites can be used to subvert the exclusionary practices employed by museums and provide tools for marginalized peoples --. The purpose of this notebook is to act as a digital tool for real life change, and it is focused on Dan Hick's [Tweet](https://twitter.com/profdanhicks/status/1375421209265983488) and book, *The Brutish Museum*.

In [1]:
%%html
<iframe src="https://drive.google.com/file/d/1txSH3UkjJgLTeQW47MGLfrht7AHCEkGC/preview" width="640" height="480"></iframe>

As I poured over the long list of museums, the massive 

I live in Canada and work in Ottawa, and I poured over the long list of names and was confronted with the sheer number of stolen artifacts that remain missing from their rightful peoples. The appendix' list is a small fraction of the stolen artifacts found in hundreds of museums all over the world but it is a powerful point of focus. I want to create something, however small, that can give others the tools to scrape data from other museum websites and interrogate why they are the institutions holding these artifacts, what effect that has, what's being done with them, and what "ownership" is. Can anyone own art? Who has the power to decide? How do we give that power back to those who were stolen from?

I'll be examining the Benin bronze collections held in the Museum of History, Royal Ottawa Museum, Montreal Museum of Fine Arts, and the Royal Museum of British Columbia.

To learn more about the Benin bronzes and their history, a good place to start is with the 'Radicals in Conversation' podcast.

# Data
"This section will discuss the source of your data, is strengths / limitations, and will include the necessary code to load the data up. The discussion here may be brief; you will want to address the same questions in more depth in your Notebook Reflection.
It may well be that you are loading data from some other website; it could be that you’ve already downloaded, subsetted, and cleaned data from somewhere; in this case, you’d be loading the data from your own github repository or Datasette on Heroku."

## Mapping with Folium

In [8]:
import lxml

In [9]:
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

In [10]:
import pandas, os

In [11]:
os.listdir()
['.config', 'benin_bronze_locations2.csv', 'sample_data']

['.config', 'benin_bronze_locations2.csv', 'sample_data']

In [14]:
from google.colab import files
uploaded = files.upload()

Saving benin_bronze_locations2.csv to benin_bronze_locations2.csv


In [17]:
df1=pandas.read_csv('benin_bronze_locations2.csv', encoding = "ISO-8859-1", engine ='python')
df1

,Place,City,Country
0,Ulster Museum,Belfast,United Kingdom
1,Barber Institute University of Birmingham,Birmingham,United Kingdom
2,City Museum and Art Gallery,Birmingham,United Kingdom
3,Brighton Museum and Art Gallery,Brighton,United Kingdom
4,Bristol City Museum,Bristol,United Kingdom
5,Museum of Archaeology and Anthropology,Cambridge,United Kingdom
6,Jesus College,Cambridge,United Kingdom
7,Dumas-Egerton Trust,Cambridge,United Kingdom
8,National Museums Scotland,Edinburgh,United Kingdom
9,Myers Museum Eton College,Berkshire,United Kingdom


In [18]:
!pip install geopy

In [19]:
from geopy.geocoders import Nominatim

After installing geopy and importing Nominatim, I did a test to experiment what kind of data I could pull from single museum entries from my data list.

In [20]:
geolocator = Nominatim(user_agent="BENIN-BRONZES", timeout=2)

In [21]:
location = geolocator.geocode("Ulster Museum United Kingdom")

In [22]:
location

Location(Ulster Museum, Stranmillis Road, Queen's Quarter, Belfast, County Antrim, Northern Ireland, BT9 5AB, United Kingdom, (54.58228485, -5.935325683757221, 0.0))

Great! Now I have the means of finding all the relevant map information with the code. But the problem is that I have 160 data entries; doing all that manually would take a *very* long time. So, I had to find another way to process the large amount of data. I followed [this YouTube tutorial](https://www.youtube.com/watch?v=0IjdfgmWzMk) for some help when I found that the one for the class wasn't useful.

In [23]:
def find_location(row):

    place = row['place']

    location = geolocator.geocode(place)

    if location != None:
        return location.address, location.latitude, location.longitude, location.raw['importance']
    else:
        return "Not Found", "Not Found", "Not Found", "Not Found"

In [24]:
from geopy.geocoders import Nominatim

In [25]:
nom = Nominatim()

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [26]:
n=nom.geocode("London eye")
print(n.latitude, n.longitude)

51.50328025 -0.11968732228242385


Then it instructed me to add a new address collumn to my data set:

In [29]:
df1["Address"]=df1["Place"]+", "+df1["City"]+", "+df1["Country"]

In [30]:
df1

,Place,City,Country,Address
0,Ulster Museum,Belfast,United Kingdom,"Ulster Museum, Belfast, United Kingdom"
1,Barber Institute University of Birmingham,Birmingham,United Kingdom,"Barber Institute University of Birmingham, Birmingham, United Kingdom"
2,City Museum and Art Gallery,Birmingham,United Kingdom,"City Museum and Art Gallery, Birmingham, United Kingdom"
3,Brighton Museum and Art Gallery,Brighton,United Kingdom,"Brighton Museum and Art Gallery, Brighton, United Kingdom"
4,Bristol City Museum,Bristol,United Kingdom,"Bristol City Museum, Bristol, United Kingdom"
5,Museum of Archaeology and Anthropology,Cambridge,United Kingdom,"Museum of Archaeology and Anthropology, Cambridge, United Kingdom"
6,Jesus College,Cambridge,United Kingdom,"Jesus College, Cambridge, United Kingdom"
7,Dumas-Egerton Trust,Cambridge,United Kingdom,"Dumas-Egerton Trust, Cambridge, United Kingdom"
8,National Museums Scotland,Edinburgh,United Kingdom,"National Museums Scotland, Edinburgh, United Kingdom"
9,Myers Museum Eton College,Berkshire,United Kingdom,"Myers Museum Eton College, Berkshire, United Kingdom"


In [31]:
#Then I had to add this string to the geocode

df1["Coordinates"]=df1["Address"].apply(nom.geocode)

In [32]:
df1

,Place,City,Country,Address,Coordinates
0,Ulster Museum,Belfast,United Kingdom,"Ulster Museum, Belfast, United Kingdom","(Ulster Museum, Stranmillis Road, Queen's Quarter, Belfast, County Antrim, Northern Ireland, BT9 5AB, United Kingdom, (54.58228485, -5.935325683757221))"
1,Barber Institute University of Birmingham,Birmingham,United Kingdom,"Barber Institute University of Birmingham, Birmingham, United Kingdom",None
2,City Museum and Art Gallery,Birmingham,United Kingdom,"City Museum and Art Gallery, Birmingham, United Kingdom",None
3,Brighton Museum and Art Gallery,Brighton,United Kingdom,"Brighton Museum and Art Gallery, Brighton, United Kingdom","(Brighton Museum & Art Gallery, Church Street, Round Hill, Brighton, Brighton and Hove, South East England, England, BN1 1UB, United Kingdom, (50.823454749999996, -0.13803227795095502))"
4,Bristol City Museum,Bristol,United Kingdom,"Bristol City Museum, Bristol, United Kingdom",None
5,Museum of Archaeology and Anthropology,Cambridge,United Kingdom,"Museum of Archaeology and Anthropology, Cambridge, United Kingdom","(Museum of Archaeology and Anthropology, Downing Street, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB2 3DZ, United Kingdom, (52.202799, 0.1209726))"
6,Jesus College,Cambridge,United Kingdom,"Jesus College, Cambridge, United Kingdom","(Jesus College, Victoria Avenue, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB1 1LB, United Kingdom, (52.2084959, 0.1270643))"
7,Dumas-Egerton Trust,Cambridge,United Kingdom,"Dumas-Egerton Trust, Cambridge, United Kingdom",None
8,National Museums Scotland,Edinburgh,United Kingdom,"National Museums Scotland, Edinburgh, United Kingdom","(National War Museum, King's Stables Road, Old Town, City of Edinburgh, Scotland, EH1 2JY, United Kingdom, (55.94885295, -3.2019126171556156))"
9,Myers Museum Eton College,Berkshire,United Kingdom,"Myers Museum Eton College, Berkshire, United Kingdom",None


In [33]:
#Set values to latitudes and longitudes

In [34]:
df1["Latitude"]=df1["Coordinates"].apply(lambda x: x.latitude if x !=None else None)

In [35]:
df1["Longitude"]=df1["Coordinates"].apply(lambda x: x.longitude if x !=None else None)

In [36]:
df1

,Place,City,Country,Address,Coordinates,Latitude,Longitude
0,Ulster Museum,Belfast,United Kingdom,"Ulster Museum, Belfast, United Kingdom","(Ulster Museum, Stranmillis Road, Queen's Quarter, Belfast, County Antrim, Northern Ireland, BT9 5AB, United Kingdom, (54.58228485, -5.935325683757221))",54.582285,-5.935326
1,Barber Institute University of Birmingham,Birmingham,United Kingdom,"Barber Institute University of Birmingham, Birmingham, United Kingdom",None,NaN,NaN
2,City Museum and Art Gallery,Birmingham,United Kingdom,"City Museum and Art Gallery, Birmingham, United Kingdom",None,NaN,NaN
3,Brighton Museum and Art Gallery,Brighton,United Kingdom,"Brighton Museum and Art Gallery, Brighton, United Kingdom","(Brighton Museum & Art Gallery, Church Street, Round Hill, Brighton, Brighton and Hove, South East England, England, BN1 1UB, United Kingdom, (50.823454749999996, -0.13803227795095502))",50.823455,-0.138032
4,Bristol City Museum,Bristol,United Kingdom,"Bristol City Museum, Bristol, United Kingdom",None,NaN,NaN
5,Museum of Archaeology and Anthropology,Cambridge,United Kingdom,"Museum of Archaeology and Anthropology, Cambridge, United Kingdom","(Museum of Archaeology and Anthropology, Downing Street, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB2 3DZ, United Kingdom, (52.202799, 0.1209726))",52.202799,0.120973
6,Jesus College,Cambridge,United Kingdom,"Jesus College, Cambridge, United Kingdom","(Jesus College, Victoria Avenue, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB1 1LB, United Kingdom, (52.2084959, 0.1270643))",52.208496,0.127064
7,Dumas-Egerton Trust,Cambridge,United Kingdom,"Dumas-Egerton Trust, Cambridge, United Kingdom",None,NaN,NaN
8,National Museums Scotland,Edinburgh,United Kingdom,"National Museums Scotland, Edinburgh, United Kingdom","(National War Museum, King's Stables Road, Old Town, City of Edinburgh, Scotland, EH1 2JY, United Kingdom, (55.94885295, -3.2019126171556156))",55.948853,-3.201913
9,Myers Museum Eton College,Berkshire,United Kingdom,"Myers Museum Eton College, Berkshire, United Kingdom",None,NaN,NaN


In [37]:
!pip install folium

In [38]:
import folium

In [39]:
beninbronze_map = folium.Map(location=[6.3350, 5.6037], zoom_start=7)
beninbronze_map

In [40]:
def create_map_markers(row, beninbronze_map):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(beninbronze_map)

In [41]:
folium.Marker(location=[6.3350, 5.6037], popup="Send the bronzes home").add_to(beninbronze_map)

In [42]:
beninbronze_map

In [43]:
def create_map_markers(row, beninbronze_map):
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Place']).add_to(beninbronze_map)

In [44]:
df1.dropna(subset = ["Latitude"], inplace=True)

In [45]:
df1.dropna(subset = ["Longitude"], inplace=True)

In [46]:
nan_value = float("NaN")
df1.replace("",nan_value, inplace=True)
df1.dropna(subset = ["Latitude"], inplace=True)
df1.dropna(subset = ["Longitude"], inplace=True)

In [47]:
df1

,Place,City,Country,Address,Coordinates,Latitude,Longitude
0,Ulster Museum,Belfast,United Kingdom,"Ulster Museum, Belfast, United Kingdom","(Ulster Museum, Stranmillis Road, Queen's Quarter, Belfast, County Antrim, Northern Ireland, BT9 5AB, United Kingdom, (54.58228485, -5.935325683757221))",54.582285,-5.935326
3,Brighton Museum and Art Gallery,Brighton,United Kingdom,"Brighton Museum and Art Gallery, Brighton, United Kingdom","(Brighton Museum & Art Gallery, Church Street, Round Hill, Brighton, Brighton and Hove, South East England, England, BN1 1UB, United Kingdom, (50.823454749999996, -0.13803227795095502))",50.823455,-0.138032
5,Museum of Archaeology and Anthropology,Cambridge,United Kingdom,"Museum of Archaeology and Anthropology, Cambridge, United Kingdom","(Museum of Archaeology and Anthropology, Downing Street, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB2 3DZ, United Kingdom, (52.202799, 0.1209726))",52.202799,0.120973
6,Jesus College,Cambridge,United Kingdom,"Jesus College, Cambridge, United Kingdom","(Jesus College, Victoria Avenue, Petersfield, Cambridge, Cambridgeshire, East of England, England, CB1 1LB, United Kingdom, (52.2084959, 0.1270643))",52.208496,0.127064
8,National Museums Scotland,Edinburgh,United Kingdom,"National Museums Scotland, Edinburgh, United Kingdom","(National War Museum, King's Stables Road, Old Town, City of Edinburgh, Scotland, EH1 2JY, United Kingdom, (55.94885295, -3.2019126171556156))",55.948853,-3.201913
10,Glasgow Art Gallery,Glasgow,United Kingdom,"Glasgow Art Gallery, Glasgow, United Kingdom","(Hunterian Art Gallery, Hillhead Street, Yorkhill, Hillhead, Glasgow, Glasgow City, Scotland, G12 8QF, United Kingdom, (55.8730964, -4.2887702))",55.873096,-4.288770
14,Hull Museums,Hull,United Kingdom,"Hull Museums, Hull, United Kingdom","(Hands on History Museum, South Church Side, Old Town, Hull, Kingston upon Hull, Yorkshire and the Humber, England, HU1 1RR, United Kingdom, (53.74154245, -0.33549222175794247))",53.741542,-0.335492
16,Leeds City Museum,Leeds,United Kingdom,"Leeds City Museum, Leeds, United Kingdom","(Leeds City Museum, Millennium Square, Woodhouse, Leeds, West Yorkshire, Yorkshire and the Humber, England, LS2 8BH, United Kingdom, (53.80158325, -1.5468995820159708))",53.801583,-1.546900
17,Royal Armouries,Leeds,United Kingdom,"Royal Armouries, Leeds, United Kingdom","(Royal Armouries, Armouries Drive, Steander, Richmond Hill, Leeds, West Yorkshire, Yorkshire and the Humber, England, LS10 1LG, United Kingdom, (53.79183725, -1.5322648752506947))",53.791837,-1.532265
18,World Museums,Liverpool,United Kingdom,"World Museums, Liverpool, United Kingdom","(World Museum Liverpool, William Brown Street, Ropewalks, Liverpool, North West England, England, L3 8EN, United Kingdom, (53.4098342, -2.9818017))",53.409834,-2.981802


In [48]:
df1.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], 
                                                  row["Longitude"]]).add_to(beninbronze_map),
         axis=1)
beninbronze_map

In [49]:
beninbronze_map.save("stolen-bronzes-map.html")

# Conclusion

# References

Abt, Jeffrey. “The Origins of the Public Museum.” In A Companion to Museum Studies, 115–134. Malden, MA, USA: Blackwell Publishing Ltd, 2006.

Bennett, Tony. 1990. “The Political Rationality of the Museum,” Continuum: The Australian Journal of Media and Culture 2, no. 1 (1990).

Hudson, David J. 2016. “On Dark Continents and Digital Divides: Information Inequality and the Reproduction of Racial Otherness in Library and Information Studies” https://atrium.lib.uoguelph.ca/xmlui/handle/10214/9862.

Kreps, Christina. 2008. ‘Non-western Models of Museums and Curation in Cross-cultural Perspective’in Sharon Macdonald, ed. ‘Companion to Museum Studies’.

Bivens, Joy, and Ben Garcia, Porchia Moore, nikhil trivedi, Aletheia Wittman. 2019. ‘Collections: How We Hold the Stuff We Hold in Trust’ in MASSAction, Museums As Site for Social Action, toolkit, https://static1.squarespace.com/static/58fa685dff7c50f78be5f2b2/t/59dcdd27e5dd5b5a1b51d9d8/1507646780650/TOOLKIT_10_2017.pdf

MacDonald, Sharon. 2008. “Collecting Practices” in Sharon Macdonald, ed. ‘Companion to Museum Studies’.

Sentance, Nathan mudyi. 2018. “Why Do We Collect,” Archival Decolonist blog, August 18, 2018, https://archivaldecolonist.com/2018/08/18/why-do-we-collect/

## Recommended Reading

https://www.danhicks.uk/brutishmuseums

https://www.plutobooks.com/blog/podcast-brutish-museums-benin-bronzes-decolonisation/